# Lesson 6 - Wrapping a Smolagents Agent into an ACP Server

You will now create another ACP agent built with Smolagents. This second agent will be able to search the web to handle health based questions for patients.  You will wrap the agent definition in an ACP server similarly to how you did with the first RAG agent.

## 6.1. Wrap the Agent in ACP Server

To define your agent, you will use [CodeAgent](https://smolagents.org/docs/agents-guided-tour/) from the Smolagents library. This type of agent writes and executes Python code at every step. For this agent, you will use two tools provided by Smolagents:

- DuckDuckGoSearchTool: performs a web search using DuckDuckGo browser
- VisitWebpageTool: can visit and extract content from web pages

The agent is wrapped in an ACP server using `@server.agent()` decorator. The server will also run locally using a different port number: 8001. Run the following cell to save the agent as `smolagents_server.py` under `my_acp_project`.

In [ ]:
%%writefile ../my_acp_project/smolagents_server.py
from collections.abc import AsyncGenerator
from acp_sdk.models import Message, MessagePart
from acp_sdk.server import Context, RunYield, RunYieldResume, Server
from smolagents import CodeAgent, DuckDuckGoSearchTool, LiteLLMModel, VisitWebpageTool
import logging 
from dotenv import load_dotenv

load_dotenv() 

server = Server()

model = LiteLLMModel(
    model_id="openai/gpt-4",  
    max_tokens=2048
)

@server.agent()
async def health_agent(input: list[Message], context: Context) -> AsyncGenerator[RunYield, RunYieldResume]:
    "This is a CodeAgent which supports the hospital to handle health based questions for patients. Current or prospective patients can use it to find answers about their health and hospital treatments."
    agent = CodeAgent(tools=[DuckDuckGoSearchTool(), VisitWebpageTool()], model=model)

    prompt = input[0].parts[0].content
    response = agent.run(prompt)

    yield Message(parts=[MessagePart(content=str(response))])


if __name__ == "__main__":
    server.run(port=8000)

## 6.2. Run the Hospital ACP Server

Now to activate your configured ACP agent, you also need to run your agent from another terminal.

- Open the second terminal by running the cell below;
- Type `uv run smolagents_server.py` to run the server and activate your ACP agent.

Please see note below if you want to replicate the work locally on your machine.

In [ ]:
from IPython.display import IFrame
import os
url = os.environ.get('DLAI_LOCAL_URL').format(port=8888)
IFrame(f"{url}terminals/2", width=800, height=600)

If you see this warning: 
`WARNING: Can not reach server, check if running on http://127.0.0.1:8333 : Request failed after 5 retries`
you can ignore it. You'll learn later in another lesson about the BeeAI platform, which a registry you can use to manage and discover agents. If the platform is installed, it runs by default on port 8333. The ACP servers are configured to automatically connect to the platform. Since the platform is not installed in this environment, the ACP server will generate a warning.

**Note: How to update the `my_acp_project` locally on your machine so you can run the second server?**
- cd `my_acp_project`
- `uv add smolagents duckduckgo-search markdownify requests`

## 6.3. Resources

- [Smolagents doc](https://smolagents.org/docs/smolagent-docs/)
- [Short course: Building Code Agents with Hugging Face Smolagents ](https://www.deeplearning.ai/short-courses/building-code-agents-with-hugging-face-smolagents/)
- [Same code using a local open source model: `ollama_chat/qwen2.5:14b`](https://github.com/nicknochnack/ACPWalkthrough/blob/main/4.%20smolagents%20ACP.py)

<p style="background-color:#fff6ff; padding:15px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px"> 💻 &nbsp; <b>To access the <code>my_acp_project</code> folder:</b> 1) click on the <em>"File"</em> option on the top menu of the notebook and then 2) click on <em>"Open"</em>. 

<div style="background-color:#fff6ff; padding:13px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px">
<p> ⬇ &nbsp; <b>Download Notebooks:</b> 1) click on the <em>"File"</em> option on the top menu of the notebook and then 2) click on <em>"Download as"</em> and select <em>"Notebook (.ipynb)"</em>.</p>

</div>